In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data
import logging
from gensim.models import word2vec

In [2]:
# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [3]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
 nltk.download('punkt')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

[nltk_data] Downloading package punkt to /home/nharpale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/bs4/__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/bs4/__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/bs4/__init__.py:335: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/bs4/__init__.py:335: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/bs4/__init__.py:335: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP c

In [6]:
 print(len(sentences))

795538


In [7]:
print(sentences[0])
print(sentences[1])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [8]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)

print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-02-22 22:59:11,702 : INFO : collecting all words and their counts
2019-02-22 22:59:11,703 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-22 22:59:11,747 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-02-22 22:59:11,796 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-02-22 22:59:11,844 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types
2019-02-22 22:59:11,894 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types


Training model...


2019-02-22 22:59:11,939 : INFO : PROGRESS: at sentence #50000, processed 1116962 words, keeping 37761 word types
2019-02-22 22:59:11,981 : INFO : PROGRESS: at sentence #60000, processed 1338403 words, keeping 40723 word types
2019-02-22 22:59:12,024 : INFO : PROGRESS: at sentence #70000, processed 1561579 words, keeping 43333 word types
2019-02-22 22:59:12,068 : INFO : PROGRESS: at sentence #80000, processed 1780886 words, keeping 45714 word types
2019-02-22 22:59:12,111 : INFO : PROGRESS: at sentence #90000, processed 2004995 words, keeping 48135 word types
2019-02-22 22:59:12,154 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-02-22 22:59:12,197 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2019-02-22 22:59:12,240 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keeping 54119 word types
2019-02-22 22:59:12,284 : INFO : PROGRESS: at sentence #130000, processed 2894303 words, keep

2019-02-22 22:59:15,150 : INFO : PROGRESS: at sentence #770000, processed 17217946 words, keeping 121704 word types
2019-02-22 22:59:15,196 : INFO : PROGRESS: at sentence #780000, processed 17448092 words, keeping 122403 word types
2019-02-22 22:59:15,241 : INFO : PROGRESS: at sentence #790000, processed 17675168 words, keeping 123067 word types
2019-02-22 22:59:15,266 : INFO : collected 123505 word types from a corpus of 17798269 raw words and 795538 sentences
2019-02-22 22:59:15,267 : INFO : Loading a fresh vocabulary
2019-02-22 22:59:15,328 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123505, drops 107015)
2019-02-22 22:59:15,329 : INFO : effective_min_count=40 leaves 17239123 word corpus (96% of original 17798269, drops 559146)
2019-02-22 22:59:15,373 : INFO : deleting the raw counts dictionary of 123505 items
2019-02-22 22:59:15,378 : INFO : sample=0.001 downsamples 48 most-common words
2019-02-22 22:59:15,378 : INFO : downsampling leaves estimated 1

2019-02-22 23:00:06,953 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-02-22 23:00:06,956 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-22 23:00:06,968 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-22 23:00:06,970 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-22 23:00:06,971 : INFO : EPOCH - 4 : training on 17798269 raw words (12749677 effective words) took 12.9s, 989967 effective words/s
2019-02-22 23:00:07,981 : INFO : EPOCH 5 - PROGRESS: at 7.66% examples, 969762 words/s, in_qsize 8, out_qsize 0
2019-02-22 23:00:08,993 : INFO : EPOCH 5 - PROGRESS: at 15.62% examples, 983289 words/s, in_qsize 8, out_qsize 0
2019-02-22 23:00:09,995 : INFO : EPOCH 5 - PROGRESS: at 23.43% examples, 983654 words/s, in_qsize 7, out_qsize 0
2019-02-22 23:00:10,996 : INFO : EPOCH 5 - PROGRESS: at 31.26% examples, 985726 words/s, in_qsize 8, out_qsize 0
2019-02-22 23:00:12,000 : INFO : EPOCH 5 - PR

In [9]:
model.doesnt_match("man woman child kitchen".split())

/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [10]:
model.doesnt_match("france england germany berlin".split())

/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [11]:
model.doesnt_match("paris berlin london austria".split())

/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [12]:
model.most_similar("man")

/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6283491253852844),
 ('lady', 0.6065947413444519),
 ('lad', 0.5885618925094604),
 ('millionaire', 0.5374081134796143),
 ('monk', 0.537163496017456),
 ('farmer', 0.5232622623443604),
 ('men', 0.5139851570129395),
 ('businessman', 0.5135544538497925),
 ('person', 0.5050943493843079),
 ('boy', 0.5006979703903198)]

In [13]:
model.most_similar("queen")

/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.6832795739173889),
 ('bride', 0.6255319714546204),
 ('belle', 0.6147541999816895),
 ('maid', 0.6069222688674927),
 ('eva', 0.5902668237686157),
 ('victoria', 0.5833011865615845),
 ('latifah', 0.5822697877883911),
 ('mistress', 0.582189679145813),
 ('duchess', 0.5802056789398193),
 ('goddess', 0.5793904066085815)]

In [14]:
model.most_similar("awful")

/home/nharpale/PA1/myenv1/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7702475786209106),
 ('horrible', 0.7318698167800903),
 ('atrocious', 0.7288468480110168),
 ('abysmal', 0.7017455697059631),
 ('dreadful', 0.6945229768753052),
 ('horrendous', 0.6807694435119629),
 ('horrid', 0.6747561693191528),
 ('appalling', 0.6658003330230713),
 ('lousy', 0.622587263584137),
 ('amateurish', 0.6204423308372498)]